### Stealing a bitcoin transaction — 100

We are given the Bitcoin testnet transaction hash:
[23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3](https://blockstream.info/testnet/tx/23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3)
* Goal: We want to know how to spent this tBTC output.
* Hint: The unlocking script (scriptSig) can be find in 8 Byte of HEX and can be converted to Cleartext ASCII.

Hints:
1. Find the SCRIPTPUBKEY that locks the output value
2. Understand how the input must look like, to unlock this locking script.
3. HINT: check the adress where this transaction comes from.
4. HINT2: find other adresses that received transactions from the initial adress and see if the adress already unlocked the same UTXO on a different transaction and used the same unlock script (scriptsig)



In [ ]:
# 1. BTC Transaction in 23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3
# Hint: https://en.bitcoin.it/wiki/Script
# https://opcodeexplained.com/script/asm.html

In [15]:
from blockcypher import get_transaction_details

# Transaction hash
tx_hash = "23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3"

# Fetch transaction details
tx_details = get_transaction_details(tx_hash, coin_symbol='btc-testnet')

# Extract the output script
for output in tx_details['outputs']:
    script_asm = output['script']
    print(f"Output Script (ASM): {script_asm}")
    print(f"Removed OPCodes: {script_asm.replace('aa20', '')}")

# OP_HASH256 OP_PUSHBYTES_32 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f
# This script means that to spend this output, you need to provide an input that, when hashed twice using SHA-256, matches the given 32-byte hash.


Output Script (ASM): aa2064c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f87
Removed OPCodes: 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f87


In [ ]:
# Task 2: 
# Next step:
# We know the output: OP_HASH256 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f OP_EQUAL
# This means you need to find an input that, when hashed twice with SHA-256, have to equal 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f.


#  2.1 Next step Get the funding adress
#  Look at the transactions of the address that funded the given transaction. This might help you find a related transaction with a custom script that could contain the input you need.


# 2.2 Check Similar Scripts
# Look for transactions with similar scripts. If you find a transaction with the same script, it might contain the input you're looking for.

In [20]:
# 2.3 Decode
# If you find a transaction with the same script, decode the input to see if it matches the desired value when double-hashed with SHA-256.


065303d56ca16a47959b843922bb8179014eab6b8ef7ee0608b2b39cac963feb


In [2]:
# Convert the SCRIPTSIG to ASCII

import binascii

hex_string = "636f696e62617365"
decoded_string = binascii.unhexlify(hex_string).decode('utf-8')
print(decoded_string)

# Alternative https://www.rapidtables.com/convert/number/hex-to-ascii.html

coinbase


### Solution


We are given the Bitcoin testnet transaction hash 

https://blockstream.info/testnet/tx/23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3
23c9470a2269cf6430569afdd3e2e8f35f633b6cdbc34107ddbab932c4146ba3 

and we have to find how to spend the output.

The output script is OP_HASH256 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f OP_EQUAL, so we need to find an input which gives the desired value when double hashed with sha256.

Several possibilities : 
* try to guess the password or bruteforce it
* simply look at the transactions of the funding address
  * find interesting ones coming from custom scripts. And one is the same locking script, so the flag is in this transaction encoded in hex


https://blockstream.info/testnet/address/my3zveqVpM4KJQtGBxeqEXH6uawkpNx7mG
send also this tx:
https://blockstream.info/testnet/tx/d2fb9169cf369657f7ef16569f98c5666dc8d53b09e84cf0d6f46b40dad67936
SCRIPTSIG (ASM)	OP_PUSHBYTES_8 636f696e62617365
SCRIPTSIG (HEX)	08636f696e62617365
NSEQUENCE	0xffffffff
PREVIOUS OUTPUT SCRIPT	OP_HASH256 OP_PUSHBYTES_32 64c9e17ec60cd8e074aa24fdb533b3147b651cc18d7c9bcd6b00ef483affc22f OP_EQUAL (unknown)



This Transaction pushes 8 Byte onto the Chain 636f696e62617365

When you convert this HEX to ASCII it gives you the flag coinbase